In [4]:
import psycopg2
import pandas as pd
import json

C:\Users\valen\AppData\Local\Temp\ipykernel_10236\1122502516.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
def connect_to_db():
    db_conn = None
    try:
        with open('C:/Users/valen/OneDrive/Documentos/workshop_1/Workshop_1/loading_to_postgres/db_config.json', 'r') as config_file:
            db_settings = json.load(config_file)

        db_conn = psycopg2.connect(
            host='localhost',
            user=db_settings['user'],
            password=db_settings['password'],
            dbname=db_settings['database']
        )
        print('Connection to the database was successful')
    except psycopg2.DatabaseError as db_error:
        print('Failed to connect to the database:', db_error)
    return db_conn

connect_to_db()

Connection to the database was successful


<connection object at 0x000001CBDCA85F10; dsn: 'user=postgres password=xxx dbname=workshop3 host=localhost', closed: 0>

In [7]:
def get_hires_by_year(db_conn):
    # Creamos un objeto cursor utilizando la conexión a la base de datos
    cursor = db_conn.cursor()
    
    # Definimos la consulta SQL para contar los contratados agrupados por año
    query = """
    SELECT EXTRACT(YEAR FROM date_of_application) AS year, COUNT(*) 
    FROM applicants 
    WHERE hired_status = 'True' 
    GROUP BY year
    ORDER BY year;
    """
    
    try:
        # Ejecutamos la consulta
        cursor.execute(query)
        # Obtenemos todos los resultados de la consulta
        hires_by_year = cursor.fetchall()
        # Creamos un DataFrame a partir de los resultados
        df_hires_by_year = pd.DataFrame(hires_by_year, columns=['Year', 'Hires'])
        return df_hires_by_year
    except psycopg2.DatabaseError as e:
        print("Database error:", e)
    finally:
        # Cerramos el cursor
        cursor.close()

# Ahora llamamos a la función 'get_hires_by_year' usando la conexión establecida
db_conn = connect_to_db()
if db_conn:
    hires_by_year_df = get_hires_by_year(db_conn)
    db_conn.close()  # Aseguramos cerrar la conexión
    print(hires_by_year_df)
else:
    print("No se pudo establecer una conexión con la base de datos.")



Connection to the database was successful
   Year  Hires
0  2018   1409
1  2019   1524
2  2020   1485
3  2021   1485
4  2022    795
